In [3]:
from paddlenlp.transformers import ErnieModel,ErnieTokenizer
from model import ErnieGRUCRF
from data import load_dataset,load_dict_single

In [5]:
train_ds, dev_ds = load_dataset(datafiles=('./data/train.0', './data/dev.0'))

# 把类别名转为id
label_vocab = load_dict_single('./data/tag.txt')

# 查看训练集和测试集的大小
print("训练集大小:", len(train_ds))
print("测试集集大小:", len(dev_ds))
print(train_ds[0])
print(dev_ds[0])
print(label_vocab)  # 57个分类

训练集大小: 67624
测试集集大小: 27715
(['浙', '江', '省', '杭', '州', '市'], ['B-prov', 'I-prov', 'E-prov', 'B-city', 'I-city', 'E-city'])
(['浙', '江', '省', '嗯', ','], ['B-prov', 'I-prov', 'E-prov', 'O', 'O'])
{'O': 0, 'B-prov': 1, 'I-prov': 2, 'E-prov': 3, 'B-city': 4, 'I-city': 5, 'E-city': 6, 'B-district': 7, 'I-district': 8, 'E-district': 9, 'B-town': 10, 'I-town': 11, 'E-town': 12, 'B-community': 13, 'I-community': 14, 'E-community': 15, 'B-poi': 16, 'E-poi': 17, '': 18, 'B-road': 19, 'E-road': 20, 'B-roadno': 21, 'I-roadno': 22, 'E-roadno': 23, 'I-poi': 24, 'B-assist': 25, 'E-assist': 26, 'B-distance': 27, 'E-distance': 28, 'I-road': 29, 'B-intersection': 30, 'I-intersection': 31, 'E-intersection': 32, 'S-assist': 33, 'B-subpoi': 34, 'I-subpoi': 35, 'E-subpoi': 36, 'B-devzone': 37, 'I-devzone': 38, 'E-devzone': 39, 'B-houseno': 40, 'I-houseno': 41, 'E-houseno': 42, 'I-distance': 43, 'B-road2': 44, 'I-road2': 45, 'E-road2': 46, 'I-assist': 47, 'B-cellno': 48, 'I-cellno': 49, 'E-cellno': 50, 'B-floo

In [6]:
pretrained_model = ErnieModel.from_pretrained(r"ernie-3.0-medium-zh")
tokenizer = ErnieTokenizer.from_pretrained("ernie-3.0-medium-zh")

[2024-03-05 17:35:29,554] [    INFO] - Already cached C:\Users\huangkai\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-03-05 17:35:29,556] [    INFO] - Loading weights file model_state.pdparams from cache at C:\Users\huangkai\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-03-05 17:35:30,138] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-03-05 17:35:40,208] [ WARNING] - Some weights of the model checkpoint at ernie-3.0-medium-zh were not used when initializing ErnieModel: ['ernie.encoder.layers.6.norm1.bias', 'ernie.encoder.layers.6.self_attn.k_proj.bias', 'ernie.encoder.layers.6.self_attn.v_proj.weight', 'ernie.encoder.layers.6.linear2.weight', 'ernie.encoder.layers.6.self_attn.k_proj.weight', 'ernie.encoder.layers.6.self_attn.q_proj.weight', 'ernie.encoder.layers.6.linear1.bias', 'ernie.encoder.layers.6.self_attn.q_proj.bias', 'ernie.encoder.layers.6.self_attn.v_proj.bias', 'ernie.encoder.layers.6.self_attn.out_proj.wei

In [9]:
model = ErnieGRUCRF(pretrained_model, 300, len(label_vocab), 100)
for params in model.gru.parameters():
    print(params)

Parameter containing:
Tensor(shape=[900, 768], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.04332657, -0.04441625, -0.01207554, ..., -0.05673484,
         -0.03251170,  0.04750285],
        [-0.00657710,  0.01513232, -0.02861152, ...,  0.04236169,
         -0.05304147,  0.01019591],
        [ 0.00048666, -0.00581980,  0.02790153, ..., -0.05048276,
          0.05324689,  0.05744588],
        ...,
        [ 0.04652891,  0.01705701,  0.02377181, ..., -0.01949210,
          0.00191563, -0.00405062],
        [ 0.00107701, -0.02988624, -0.05081275, ...,  0.01962170,
         -0.01606600,  0.00609083],
        [-0.02507464, -0.04001677,  0.05044233, ...,  0.04507089,
         -0.02034718, -0.05679528]])
Parameter containing:
Tensor(shape=[900, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.00772423, -0.03316343,  0.02157742, ...,  0.05252570,
         -0.00408141,  0.00475048],
        [-0.00472074,  0.01936758,  0.04530606, ...,  0.05587174,
    

In [10]:
import paddle
state_dict = paddle.load("ner_model/model_42280.pdparams")
model.set_dict(state_dict)

(['fc.weight',
  'fc.bias',
  'crf.transitions',
  'crf_loss.crf.transitions',
  'viterbi_decoder.transitions'],
 ['fc.weight',
  'fc.bias',
  'crf.transitions',
  'crf_loss.crf.transitions',
  'viterbi_decoder.transitions'])

In [11]:
for params in model.gru.parameters():
    print(params)

Parameter containing:
Tensor(shape=[900, 768], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.00431278,  0.00782988, -0.01447922, ..., -0.03163040,
          0.02313145,  0.03078911],
        [-0.04491691, -0.00364067, -0.00983983, ..., -0.01720943,
          0.07097724, -0.00365569],
        [-0.06247249,  0.01491211, -0.04569069, ..., -0.02207063,
         -0.06114850,  0.04144102],
        ...,
        [-0.01824863,  0.01792793, -0.05014712, ...,  0.00244747,
          0.03399225, -0.03012270],
        [ 0.04483406, -0.07919921,  0.01683323, ..., -0.05874594,
         -0.06198655,  0.01496401],
        [-0.05167799,  0.03493969,  0.01011887, ...,  0.05697599,
          0.01123167, -0.04701636]])
Parameter containing:
Tensor(shape=[900, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.01338924, -0.04375196,  0.00523560, ...,  0.04217119,
         -0.00599962,  0.04805801],
        [-0.04411605, -0.04818212, -0.01897270, ...,  0.01614543,
    